<a href="https://colab.research.google.com/github/Dinesh-Adhithya-H/Deep_Learning_using_Tensorflow/blob/main/C3_2_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.0


In [2]:
!pip install -q tensorflow-datasets

In [3]:
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDHSTNE/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDHSTNE/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDHSTNE/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np
train_data,test_data=imdb["train"],imdb["test"]
testing_sequences=[]
training_sequences=[]
training_labels=[]
testing_labels=[]

In [5]:
for s,l in train_data:
  training_sequences.append(s.numpy().decode("utf8"))
  training_labels.append(l.numpy())
for s,l in test_data:
  testing_sequences.append(s.numpy().decode("utf8"))
  testing_labels.append(l.numpy())
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [6]:
vocab_size=10000
embedding_dim=20
max_length=120
trunc_type="post"
oov_tok="<OOV>"

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sequences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sequences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sequences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

In [8]:
reverse_word_index=dict([(v,k) for (k,v) in word_index.items()])
def decode_review(text):
  return " ".join([reverse_word_index.get(i,"?") for i in text])

In [9]:
print(decode_review(padded[2]))

mann photographs the <OOV> rocky mountains in a superb fashion and jimmy stewart and walter brennan give enjoyable performances as they always seem to do br br but come on hollywood a <OOV> telling the people of dawson city <OOV> to <OOV> themselves a <OOV> yes a <OOV> and to <OOV> the law themselves then <OOV> battling it out on the streets for control of the town br br nothing even remotely resembling that happened on the canadian side of the border during the <OOV> gold rush mr mann and company appear to have mistaken dawson city for <OOV> the canadian north for the american wild west br br canadian viewers be prepared for a <OOV> madness type of enjoyable


In [10]:
print(training_sequences[2])

Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.


In [11]:
model=tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6,activation="relu"),
  tf.keras.layers.Dense(1,activation="relu")                        
])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 20)           200000    
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 14406     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 214,413
Trainable params: 214,413
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(padded,training_labels_final,epochs=7,validation_data=(testing_padded,testing_labels_final))

Epoch 1/7
782/782 [==============================] - 6s 6ms/step - loss: 0.9294 - accuracy: 0.6018 - val_loss: 0.4298 - val_accuracy: 0.8161
Epoch 2/7
782/782 [==============================] - 4s 6ms/step - loss: 0.3348 - accuracy: 0.8843 - val_loss: 0.4500 - val_accuracy: 0.8276
Epoch 3/7
782/782 [==============================] - 4s 6ms/step - loss: 0.1842 - accuracy: 0.9546 - val_loss: 0.6088 - val_accuracy: 0.8333
Epoch 4/7
782/782 [==============================] - 5s 6ms/step - loss: 0.0874 - accuracy: 0.9920 - val_loss: 0.8141 - val_accuracy: 0.8309
Epoch 5/7
782/782 [==============================] - 4s 6ms/step - loss: 0.0374 - accuracy: 0.9973 - val_loss: 0.9246 - val_accuracy: 0.8284
Epoch 6/7
782/782 [==============================] - 4s 6ms/step - loss: 0.0556 - accuracy: 0.9909 - val_loss: 1.3172 - val_accuracy: 0.8008
Epoch 7/7
782/782 [==============================] - 4s 6ms/step - loss: 0.0739 - accuracy: 0.9883 - val_loss: 1.3025 - val_accuracy: 0.8050


In [13]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

(10000, 20)


In [14]:
import io
out_v=io.open("vecs2.tsv","w",encoding="utf-8")
out_m=io.open("meta2.tsv","w",encoding="utf-8")
for word_num in range(1,vocab_size):
  word=reverse_word_index[word_num]
  embeddings=weights[word_num]
  out_m.write(word+"\n")
  out_v.write("\t".join([str(x) for x in embeddings])+"\n")
out_v.close()
out_m.close()

In [15]:
len(reverse_word_index)

88583

In [16]:
try:
  from google.colab import files
except ImportError:
    pass
else:
    files.download("vecs2.tsv")
    files.download("meta2.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
sentence=" i am thinking its awesome"
sequence=tokenizer.texts_to_sequences([sentence])

In [18]:
sequence

[[11, 242, 534, 92, 1188]]